<a href="https://colab.research.google.com/github/ericslevenson/arctic-surface-water/blob/main/dilate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
author: ericslevenson
date: 10/16/2022
description: dilate lake predictions

In [ ]:
# Authenticate private account (only required for exporting to drive/gee/gcp)
from google.colab import auth 
auth.authenticate_user()
# Google Drive setup
from google.colab import drive
drive.mount('/content/drive')
# Complete the environment
!pip install rasterio
!pip install rioxarray

In [ ]:
import os
from rasterio import features
import pandas as pd
import shapely
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from rasterio.mask import mask
from rasterio import plot
from rasterio.merge import merge
from rasterio.plot import show
import rasterio
import rioxarray as rxr
import scipy

In [ ]:
predPath = '/content/drive/MyDrive/UNET_outputs/' # path to unet prediction directory
outPath = '/content/drive/MyDrive/UNET_pred_dilated/' # path to dilated prediction directory


In [ ]:
for i, f in enumerate(os.listdir(predPath)):
  print(i, f)

In [ ]:
# Option 1: Dilate lake predictions, vectorize, export shapefile
kernel = np.ones((3,3),np.uint8) # define kernel matrix for dilation
for i, f in enumerate(os.listdir(predPath)):
    # Read in image to np array
    with rasterio.open(predPath+f) as src:
      data = src.read(1, masked = True)
      meta = src.meata
      meta.update({'nodata':np.nan})
      trans = src.transform
      dilated_pred = scipy.ndimage.binary_dilation(pred, kernel, iterations = 6)
      # generate shapes
      shape_gen = ((shape(s), v) for s, v in shapes(data, transform=src.transform))

      # build a pd.DataFrame of shapes and convert to geodataframe
      df = DataFrame(shape_gen, columns=['geometry', 'class'])
      gdf = GeoDataFrame(df["class"], geometry=df.geometry, crs=src.crs)
      # filter out non-lake polygons
      gdf = gdf[gdf['class'] == 1]
      # write to file
      gdf.to_file(outPath + f.split('.')[0] + '_bufferedLakes.shp')
    print('writing ' + f + ' aka number ' + str(i+1)) 

writing 20160606_02WPU__unet_pred.tif aka number 1
writing 20160606_02WPT__unet_pred.tif aka number 2
writing 20160606_02VNR__unet_pred.tif aka number 3
writing 20160606_02VNQ__unet_pred.tif aka number 4
writing 20180614_02VMR__unet_pred.tif aka number 5
writing 20180611_04VDP__unet_pred.tif aka number 6
writing 20170714_02WPB__unet_pred.tif aka number 7
writing 20180611_04VDN__unet_pred.tif aka number 8
writing 20170714_02WPA__unet_pred.tif aka number 9
writing 20190614_02VLN__unet_pred.tif aka number 10
writing 20180613_02VPM__unet_pred.tif aka number 11
writing 20180611_04VDM__unet_pred.tif aka number 12


In [ ]:
# Option 2: Dilate lake predictions, export raster
kernel = np.ones((3,3),np.uint8) # define kernel matrix for dilation
for i, f in enumerate(os.listdir(predPath)):
    # Read in image to np array
    imgReader = rasterio.open(predPath+f)
    pred = imgReader.read(1)
    # get metadata and transform
    meta = imgReader.meta
    meta.update({'nodata':np.nan})
    trans = imgReader.transform
    # dilate prediction image
    dilated_pred = scipy.ndimage.binary_dilation(pred, kernel, iterations = 6)
    # write dilated prediction
    with rasterio.open(outPath + f.split('.')[0] + '_dilated.tif', 'w', **meta) as dest:
      dest.write(dilated_pred, 1)  
      print('writing ' + f + ' aka number ' + str(i+1) )